In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
# In this notebook, we try to fit a model of growth constant.
# The dataset consists of bended rectangles, done by applying an implicit module of order 1 with known linear growth law.
# To generate the dataset, simply execute "python3 generate_bending.py bendings.pickle nb_of_elem_in_dataset".
# We then fit a polynomial model of growth constants of order 2.

In [3]:
# Load our data
data = []
with open("bendings2.pickle", 'rb') as f:
    data = pickle.load(f)

template = data[0][0][:-1]
template = torch.unique_consecutive(template, dim=0)


true_implicit1_points = data[0][1]
true_C = data[0][2]
bendings = data[1:]

# Make sure each points in each data element are consecutively unique (needed when dealing varifold attachment atm).
dataset = [torch.unique_consecutive(target, dim=0)[:-1]  for target in list(zip(*bendings))[0]]
dataset = [target + 0.1*torch.rand(target.shape) for target in dataset]
targets = dataset[:15]

print("Dataset size: {size}".format(size=len(targets)))

Dataset size: 10


In [4]:
%matplotlib qt5

# Display our data set and template
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
for target in targets:
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), color='grey', lw=0.5)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
plt.plot(true_implicit1_points[:, 0].numpy(), true_implicit1_points[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()

In [38]:
# Defines the implicit module of order 1 that will be used by the atlas.
sigma_implicit1 = 1.5

implicit1_points = true_implicit1_points
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)
C_init = torch.ones(implicit1_points.shape[0], 2, 1)
implicit1 = dm.DeformationModules.ImplicitModule1(2, true_implicit1_points.shape[0], sigma_implicit1, C_init, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()))

# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(6, 2)
abc_init[0] = torch.ones(2)

In [39]:
# The polynomial model we will try to fit on our deformation constants
def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

In [40]:
# Creation of the atlas. Uses L2 norm attachment for now.
#atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.L2NormAttachment()], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c)
atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.VarifoldAttachment(2, [0.5, 2.])], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c)

In [41]:
# Fit the atlas
fitter = dm.Models.ModelFittingScipy(atlas, 1.)
costs = fitter.fit(targets, 150, options={'shoot_method': 'rk4', 'shoot_it': 10})

Initial energy = 3864177.500
Iteration: 1 
Total energy = 3208903.000000 
Attach cost = 3208891.000000 
Deformation cost = 12.359913
Iteration: 2 
Total energy = 2937356.500000 
Attach cost = 2937343.000000 
Deformation cost = 13.518715
Iteration: 3 
Total energy = 2741781.250000 
Attach cost = 2741757.000000 
Deformation cost = 24.341946
Iteration: 4 
Total energy = 1407384.125000 
Attach cost = 1407280.250000 
Deformation cost = 103.908348
Iteration: 5 
Total energy = 1268138.500000 
Attach cost = 1268043.250000 
Deformation cost = 95.267151
Iteration: 6 
Total energy = 1120222.000000 
Attach cost = 1120128.625000 
Deformation cost = 93.268692
Iteration: 7 
Total energy = 1045362.000000 
Attach cost = 1045272.750000 
Deformation cost = 89.134865
Iteration: 8 
Total energy = 975127.312500 
Attach cost = 975022.125000 
Deformation cost = 105.126411
Iteration: 9 
Total energy = 910152.812500 
Attach cost = 910053.687500 
Deformation cost = 99.010811
Iteration: 10 
Total energy = 882780.

Iteration: 46 
Total energy = 525577.750000 
Attach cost = 525046.375000 
Deformation cost = 531.399414
Iteration: 47 
Total energy = 525573.437500 
Attach cost = 525042.187500 
Deformation cost = 531.347900
Iteration: 48 
Total energy = 525568.000000 
Attach cost = 525037.000000 
Deformation cost = 530.953918
Iteration: 49 
Total energy = 525561.750000 
Attach cost = 525030.625000 
Deformation cost = 531.110535
Iteration: 50 
Total energy = 525554.125000 
Attach cost = 525022.687500 
Deformation cost = 531.370361
Iteration: 51 
Total energy = 525540.875000 
Attach cost = 525008.687500 
Deformation cost = 532.231506
Iteration: 52 
Total energy = 525536.875000 
Attach cost = 525004.125000 
Deformation cost = 532.800476
Iteration: 53 
Total energy = 525530.312500 
Attach cost = 524996.750000 
Deformation cost = 533.525330
Iteration: 54 
Total energy = 525524.125000 
Attach cost = 524989.687500 
Deformation cost = 534.394653
Iteration: 55 
Total energy = 525518.062500 
Attach cost = 52498

Iteration: 91 
Total energy = 522819.593750 
Attach cost = 522219.875000 
Deformation cost = 599.706177
Iteration: 92 
Total energy = 522814.375000 
Attach cost = 522214.750000 
Deformation cost = 599.642212
Iteration: 93 
Total energy = 522809.906250 
Attach cost = 522210.281250 
Deformation cost = 599.645447
Iteration: 94 
Total energy = 522806.281250 
Attach cost = 522205.625000 
Deformation cost = 600.613037
Iteration: 95 
Total energy = 522797.875000 
Attach cost = 522197.000000 
Deformation cost = 600.908142
Iteration: 96 
Total energy = 522787.937500 
Attach cost = 522186.187500 
Deformation cost = 601.776611
Iteration: 97 
Total energy = 522780.687500 
Attach cost = 522178.531250 
Deformation cost = 602.183228
Iteration: 98 
Total energy = 522755.937500 
Attach cost = 522153.062500 
Deformation cost = 602.911560
Iteration: 99 
Total energy = 522723.281250 
Attach cost = 522120.500000 
Deformation cost = 602.790771
Iteration: 100 
Total energy = 522679.500000 
Attach cost = 5220

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()

In [42]:
# Retrieve the optimised coeffcient and then compare the generated growth constraint to the true one.
abc_fit = atlas.parameters[-1].detach()
print(abc_fit)
C_fit = pol_order_2(implicit1_points, abc_fit[0].unsqueeze(1), abc_fit[1].unsqueeze(1), abc_fit[2].unsqueeze(1), abc_fit[3].unsqueeze(1), abc_fit[4].unsqueeze(1), abc_fit[5].unsqueeze(1)).t().unsqueeze(2)

# The closer to 1. it is, the better.
print(torch.dot((true_C/torch.norm(true_C)).flatten(), (C_fit/torch.norm(C_fit)).flatten()))

tensor([[ 1.8250e-01,  5.5295e+00],
        [ 3.0714e-01,  1.1693e+01],
        [-5.3471e-03,  7.9408e-02],
        [-6.8132e-02,  3.7746e-01],
        [-4.5442e-01, -1.1643e-01],
        [-4.1517e-02,  7.8651e-02]])
tensor(0.9939)


In [43]:
ax0 = plt.subplot(111, aspect='equal')
dm.Utilities.plot_C_arrow(ax0, implicit1_points, true_C, R=implicit1_R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, R=R, scale=1., color='black')

In [44]:
pts = atlas.models[0].init_manifold.gd[1][0].detach()
R = atlas.models[0].init_manifold.gd[1][1].detach()
ax1 = plt.subplot(111, aspect='equal')

dm.Utilities.plot_C_arrow(ax1, pts, true_C, R=R, scale=0.05, zorder=3, mutation_scale=10)
#dm.Utilities.plot_C_ellipse(ax, pts_implicit1, C, R=R, scale=1., color='black')

In [45]:
source = atlas.models[0].init_manifold.gd[0]

In [46]:
i=7
for i in range(10):
    shoot_method = 'rk4'
    shoot_it = 10
    silent_pos = atlas.models[i].init_manifold[0].gd.detach().clone()
    silent_mom = atlas.models[i].init_manifold[0].cotan.detach().clone()
    silent = dm.DeformationModules.SilentLandmarks(2, silent_pos.shape[0], gd=silent_pos, cotan=silent_mom)

    pts = atlas.models[i].init_manifold.gd[1][0].detach().clone()
    R = atlas.models[i].init_manifold.gd[1][1].detach().clone()
    imp1_cotan_pts = atlas.models[i].init_manifold.cotan[1][0].detach().clone()
    imp1_cotan_R = atlas.models[i].init_manifold.cotan[1][1].detach().clone()
    implicit1 = dm.DeformationModules.ImplicitModule1(2, pts.shape[0], sigma_implicit1, C_fit, nu=0.1, gd=(pts.clone().requires_grad_(), R.clone().requires_grad_()), cotan=(imp1_cotan_pts.clone().requires_grad_(), imp1_cotan_R.clone().requires_grad_()))


    h = dm.HamiltonianDynamic.Hamiltonian([silent, implicit1])
    intermediate_states, _ = dm.HamiltonianDynamic.shoot(h, shoot_it, shoot_method, intermediates=True)

    plt.figure()
    plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
    target = targets[i].detach().clone()

    t = -1
    transported_source = intermediate_states[t].gd[0].detach()
    plt.plot(transported_source[:, 0].numpy(), transported_source[:, 1].numpy(), color='red')
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), '--')

    plt.axis('equal')

In [60]:
plt.close('all')

In [114]:

points_per_side = 40

silent_points = dm.Utilities.generate_unit_square(points_per_side)
silent_points = silent_points*torch.tensor([1., 4.])
aabb = dm.Utilities.AABB.build_from_points(silent_points).scale(1.2)
implicit1_points = aabb.fill_uniform_density(20.)
implicit1_points = implicit1_points - torch.mean(implicit1_points, dim=0)
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)

def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

abc = torch.tensor([[0., 0.3], [0., 1.], [0., 0.], [0., 0.], [0., 1.], [0., -1.] ])
a = abc[0].unsqueeze(1)
b = abc[1].unsqueeze(1)
c = abc[2].unsqueeze(1)
d = abc[3].unsqueeze(1)
e = abc[4].unsqueeze(1)
f = abc[5].unsqueeze(1)
C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

#C = torch.zeros(implicit1_points.shape[0], 2, 1)
#C[:, 1, 0] = 10.*torch.abs(implicit1_points[:, 0] - torch.mean(implicit1_points, dim=0)[0] + 0.5*aabb.width)/aabb.width

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

dm.Utilities.plot_C_arrow(ax1, implicit1_points, C, R=implicit1_R, scale=1., zorder=3, mutation_scale=10)


In [115]:

def generate_bending():
    def _generate():
        silent_cotan = torch.randn_like(silent_points)
        implicit1_cotan = torch.zeros_like(implicit1_points)
        #val = 1000.*torch.rand(1) - 500.
        # val = 100.*torch.randn(1)

        # implicit1_cotan[0::7, 0] = val
        # implicit1_cotan[6::7, 0] = val
        #implicit1_cotan[6, 0] = val
        #implicit1_cotan[-1, 0] = 
        implicit1_cotan = 15.*torch.randn_like(implicit1_points)
        implicit1_cotan_R = torch.zeros_like(implicit1_R)

        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # plt.quiver(implicit1_points[:, 0].numpy(), implicit1_points[:, 1].numpy(), implicit1_cotan[:, 0].numpy(), implicit1_cotan[:, 1].numpy())
        # plt.axis('equal')
        # plt.show()

        # ax = plt.subplot()
        # plt.plot(silent_points[:, 0].numpy(), silent_points[:, 1].numpy())
        # dm.Utilities.plot_C_arrow(ax, implicit1_points, C, scale=0.1, mutation_scale=10.)
        # plt.axis('equal')
        # plt.show()

        silent = dm.DeformationModules.SilentLandmarks(2, silent_points.shape[0], gd=silent_points.clone().requires_grad_(), cotan=silent_cotan.clone().requires_grad_())

        implicit1 = dm.DeformationModules.ImplicitModule1(2, implicit1_points.shape[0], 0.5, C, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()), cotan=(implicit1_cotan.clone().requires_grad_(), implicit1_cotan_R.clone().requires_grad_()))

        with torch.autograd.no_grad():
            dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([silent, implicit1]), 10, 'rk4')

        return silent.manifold.gd.detach(), implicit1.manifold.gd[0].detach()

    while True:
        bending = _generate()
        #print(1.5*torch.norm(silent_points), " ", torch.norm(bending[0]), " ", 4.*torch.norm(silent_points))
        if (not torch.any(torch.isnan(bending[0]))) and (torch.norm(bending[0]) >= 0.*torch.norm(silent_points)) and (torch.norm(bending[0]) <= 100.*torch.norm(silent_points)):
            print("New data element added. Frobenius norm: {norm}".format(norm=torch.norm(bending[0])))
            return bending


In [118]:
tar = generate_bending()

New data element added. Frobenius norm: 23.770769119262695


In [119]:

ax1 = plt.subplot(111, aspect='equal')
plt.plot(silent_points[:,0], silent_points[:,1], '.b')
plt.plot(implicit1_points[:,0], implicit1_points[:,1], '.r')

plt.plot(tar[0][:,0], tar[0][:,1], '-g')


tensor(21.4277)